# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

This notebook demonstrates NVIDIA cuOpt's performance on a large popular academic [dataset by Gehring & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/). These problems are well studied and used as the basis for comparison for VRP research and product offerings.

**API Reference**: [cuOpt Documentation](https://docs.nvidia.com/cuopt)

## Environment Setup

First, let's check if we have a GPU available in this Colab environment.

In [ ]:
# Check for GPU
!nvidia-smi

### Install NVIDIA cuOpt and dependencies

Let's install the necessary packages for this notebook.

In [ ]:
# Install dependencies
!pip install cuopt cudf-cu12 numpy pandas scipy

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
from cuopt import routing
import cudf

### Download benchmark data

We need to download the Gehring & Homberger benchmark data file for our test.

In [ ]:
# Create directories
!mkdir -p data

# Download benchmark data
!wget https://www.sintef.no/contentassets/2b47f3c442dd4d7b99302cabaa06c5cc/homberger_1000_customer_instances.zip -O data/homberger_data.zip
!unzip -o data/homberger_data.zip -d data/

# Check if the file exists
homberger_1000_file = 'data/C1_10_1.TXT'
if not os.path.exists(homberger_1000_file):
    raise FileNotFoundError(f"Could not find {homberger_1000_file}. Please check the path.")

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Create helper functions

We need to create a utility function to parse the benchmark file format.

In [ ]:
# Utility function to parse Gehring & Homberger benchmark files
def create_from_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # Extract problem parameters
    n_vehicles = int(lines[4].strip().split()[0])
    vehicle_capacity = int(lines[5].strip().split()[0])
    
    # Parse location data
    data = []
    for i in range(9, len(lines)):
        if lines[i].strip():
            row = lines[i].strip().split()
            data.append([int(row[0]), float(row[1]), float(row[2]), int(row[3]),
                        int(row[4]), int(row[5]), int(row[6]), int(row[7])])
    
    # Create dataframe
    df = pd.DataFrame(data, columns=['id', 'xcord', 'ycord', 'demand', 
                                     'earliest_time', 'latest_time', 'service_time', 'dummy'])
    
    return df, vehicle_capacity, n_vehicles

### Problem Data
The data for this problem instance are provided via text file. Let's load the data.

In [ ]:
orders, vehicle_capacity, n_vehicles = create_from_file(homberger_1000_file)
n_locations = orders["demand"].shape[0]-1
print("Number of locations          : ", n_locations)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

# Initialize cuOpt Problem Model

In [ ]:
# Create a routing model with the necessary locations and vehicles
data_model = routing.DataModel(n_locations + 1, n_vehicles)

### Cost Matrix

In [ ]:
coords = list(zip(orders['xcord'].to_list(),
                  orders['ycord'].to_list()))

cost_matrix = distance.cdist(coords, coords, 'euclidean')
cost_matrix_df = cudf.DataFrame(cost_matrix.astype(np.float32))

### Set Cost Matrix

In [ ]:
# Add the distance matrix as our cost matrix
data_model.add_cost_matrix(cost_matrix_df)

### Set Fleet Data

In [ ]:
# All vehicles start and end at the depot (location 0)
veh_start_locations = cudf.Series([0] * n_vehicles)
veh_end_locations = cudf.Series([0] * n_vehicles)
data_model.set_vehicle_locations(veh_start_locations, veh_end_locations)

# Set vehicle capacities
vehicle_capacities = cudf.Series([vehicle_capacity] * n_vehicles, dtype=np.int32)

### Set Demand and Capacity

In [ ]:
# Convert demand to cudf Series
location_demand = cudf.Series(orders['demand'].values, dtype=np.int32)

# Add demand and capacity dimension
data_model.add_capacity_dimension("demand", location_demand, vehicle_capacities)

### Set Time Windows

In [ ]:
# Set time windows for locations
earliest_times = cudf.Series(orders['earliest_time'].values, dtype=np.int32)
latest_times = cudf.Series(orders['latest_time'].values, dtype=np.int32)
data_model.set_order_time_windows(earliest_times, latest_times)

# Set service times
service_times = cudf.Series(orders['service_time'].values, dtype=np.int32)
data_model.set_order_service_times(service_times)

### Helper functions to solve and process the output

In [ ]:
def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

### Get Optimized Results

Update solver config and test different run-time 

**1 Minute Time Limit**

Note: due to the large amount of data, network transfer time can exceed the requested solve time.

In [ ]:
# Create solver settings with 60 second time limit
solver_settings = routing.SolverSettings()
solver_settings.set_time_limit(60.0)

# Solve the problem
solution = routing.Solve(data_model, solver_settings)

# Get solution metrics
if solution.get_status() == 0:  # Success
    num_vehicles = solution.get_vehicle_count()
    solution_cost = solution.get_total_objective()
    print(f"Solution found with status: {solution.get_status()}")
    print(f"Number of vehicles used: {num_vehicles}")
    print(f"Total solution cost: {solution_cost}")
else:
    print(f"Failed to find a solution. Status: {solution.get_status()}")

In [ ]:
# Evaluation:
if solution.get_status() == 0:  # Success
    solution_eval(num_vehicles, solution_cost, best_known_solution)

**2 Minute Time Limit**

In [ ]:
# Create solver settings with 120 second time limit
solver_settings = routing.SolverSettings()
solver_settings.set_time_limit(120.0)

# Solve the problem
solution = routing.Solve(data_model, solver_settings)

# Get solution metrics
if solution.get_status() == 0:  # Success
    num_vehicles = solution.get_vehicle_count()
    solution_cost = solution.get_total_objective()
    print(f"Solution found with status: {solution.get_status()}")
    print(f"Number of vehicles used: {num_vehicles}")
    print(f"Total solution cost: {solution_cost}")
else:
    print(f"Failed to find a solution. Status: {solution.get_status()}")

In [ ]:
# Evaluation:
if solution.get_status() == 0:  # Success
    solution_eval(num_vehicles, solution_cost, best_known_solution)

### Visualize Solution (Optional)

Let's visualize one of the routes from our solution.

In [ ]:
# Install plotting libraries if needed
!pip install -q matplotlib

import matplotlib.pyplot as plt

# Check if we have a valid solution
if solution.get_status() == 0:
    # Pick a vehicle to visualize (first vehicle with a non-empty route)
    for veh_idx in range(n_vehicles):
        route = solution.get_vehicle_route(veh_idx)
        if len(route) > 2:  # More than just start and end depot
            break
            
    if len(route) > 2:
        print(f"Visualizing route for vehicle {veh_idx} with {len(route)} stops")
        
        # Get coordinates for the route
        route_x = [orders['xcord'][i] for i in route]
        route_y = [orders['ycord'][i] for i in route]
        
        # Plot
        plt.figure(figsize=(12, 10))
        
        # Plot all locations
        plt.scatter(orders['xcord'], orders['ycord'], c='gray', alpha=0.3, label='All locations')
        
        # Plot depot
        plt.scatter(orders['xcord'][0], orders['ycord'][0], c='red', s=100, label='Depot')
        
        # Plot route locations
        plt.scatter(route_x[1:-1], route_y[1:-1], c='blue', label='Route stops')
        
        # Plot route
        plt.plot(route_x, route_y, 'g-', label='Route')
        
        # Add arrows to show direction
        for i in range(len(route_x)-1):
            plt.arrow(route_x[i], route_y[i], 
                      (route_x[i+1] - route_x[i])*0.9, 
                      (route_y[i+1] - route_y[i])*0.9, 
                      head_width=3, head_length=6, fc='green', ec='green', alpha=0.7)
        
        plt.title(f'Route for Vehicle {veh_idx}')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("No non-empty routes found to visualize")
else:
    print("No valid solution to visualize")


SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.